In [5]:
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
from pyspark import Row
spark = SparkSession.builder.master("local[4]").appName("SQL_sess").getOrCreate()

row1 = Row(name='Alice', age=21)
row2 = Row(name='Bob', age=22)
df=spark.createDataFrame([row1,row2])
df.show()

df.collect()
df.count()



2

In [112]:
import os
from google.colab import drive
from pyspark.sql.functions import *


drive.mount("/content/drive", force_remount=True)

drive_dir = "/content/drive/MyDrive/ColabNotebooks/"
os.makedirs(drive_dir, exist_ok=True)
os.listdir(drive_dir)

# Liste les fichiers dans /content
print("Contenu de /content :", os.listdir('/content'))
# Si vous avez monté Google Drive :
print("Contenu de Google Drive :", os.listdir('/content/drive/MyDrive'))

films = spark.read.json("/content/drive/MyDrive/ColabNotebooks/films.json")
films.show(30)
films.count()

films.printSchema()

# projection
films.select(films.titre).show(1)
films.select(films['titre']).show(1)
films.select('titre').show(1)

# projection / selection
films.select(films['titre']) \
.where(films['nF']==1).show()

films.select(films.nF, films.titre, films.g).orderBy('titre').show(3,0)

films.select(films.titre.alias('t')).show(3,0)

films.withColumnRenamed('titre','t').show(3)


films.select(films.titre,films.g).show(4)

# exploide
fgenres=films.select(films.nF,
                     explode(films.g).alias('genre')).orderBy('nF').show(4)

# jointure
genres=films.select(films.nF,explode(films.g).alias('genre'))
genres.show(2)

films_sg=films.select(films.nF,films.titre)
films_sg.show(2)

films2 = films_sg.join(genres,films.nF==genres.nF)
films2.show(2)

# produit cartésien
films2 = films_sg.withColumnRenamed('nF', 'nF1').crossJoin(genres)
films2.show(2)
films2.count()

# aggrégations / group by
films_max=genres.groupBy('genre').max('nF').show(2)

films_cpt=genres.groupBy('genre') \
.agg(count('nF'),countDistinct('nF'))

films_cpt.show(2)


schema = """
          nU INT,
          nF LONG,
          note FLOAT,
          annee INT,
          mois INT,
          jour INT
        """
notes = spark.read.csv("/content/drive/MyDrive/ColabNotebooks/notesAMJ.csv", header='true', schema=schema)
notes.printSchema()
notes=notes.persist()
notes.count() #résultat: 100004

notes_films = notes.select(notes.nF.alias('nF2'), notes.note)
films_notes = films.join(notes_films, films.nF==notes_films.nF2)
films_notes.printSchema()


# collect_list
films_notes.groupBy('nF')\
.agg(collect_list('note')
.alias('notes'))
films_notes.show(2)

fn=films_notes.groupBy('nF')\
.agg(collect_list('note')
.alias('notes'))
fn.show(2)

# Functions
films.select(lower(films.titre)).show(3)
films.select(max(films.nF)).show(1)

films.select(films.titre).show(1,0)
films.select(split(films.titre,'[()]+')[0].alias('titre'),
             split(films.titre,'[()]+')[1].alias('annee')).show(1,0)

f2=films.withColumn('nbgenres',size(films.g))
f2.show(2)
f2.filter(f2.nbgenres>2).show(2)

# User-defined functions
@udf('float') # float: type du résultat
def mamoy(l):
  sum=0
  len=0
  for x in l:
    sum=sum+x
    len=len+1
  return sum/len

fn.show(20)
fn2=fn.withColumn('avg',mamoy(fn.notes))
fn2.show(2)

# Conversions
rdd=films.rdd
rdd.map(lambda x: x["g"]).take(5)
rdd.flatMap(lambda x: x["g"]).distinct().take(5)
rdd.toDF().select("titre").take(3)



[Row(titre='Prime of Miss Jean Brodie, The (1969)'),
 Row(titre='Lesson of the Evil (Aku no kyôten) (2012)'),
 Row(titre='Fox and the Hound, The (1981)')]